# Comikry — Colab SFX Server + Web UI

This notebook turns a Colab runtime into a **public SFX server + full Comikry backend**.

Run all cells top-to-bottom. At the end you will get:
- A `SFX_API_URL=https://...` line to paste into your local `.env` (so your local backend offloads AudioGen to this Colab GPU)
- A **clickable Web UI link** that opens the full Comikry player in your browser, already connected to this server

**Recommended runtime:** *Runtime → Change runtime type → T4 GPU*  
**API key:** left sidebar → 🔑 *Secrets* → *Add new secret* → name: `OPENROUTER_API_KEY`

In [ ]:
# ── Cell 1: Clone / update the repo ──────────────────────────────────────────
import os
from pathlib import Path

REPO_URL = 'https://github.com/LIdro/comikry.git'
REPO_DIR = Path('/content/comikry')

if REPO_DIR.exists():
    print('► Pulling latest changes …')
    os.system(f'git -C {REPO_DIR} pull --ff-only')
else:
    print('► Cloning comikry repo …')
    os.system(f'git clone --depth 1 {REPO_URL} {REPO_DIR}')

import sys
if str(REPO_DIR) not in sys.path:
    sys.path.insert(0, str(REPO_DIR))

print(f'✅  Repo ready at {REPO_DIR}')

In [ ]:
# ── Cell 2: System packages ───────────────────────────────────────────────────
# ffmpeg   — WAV→MP3 conversion after AudioGen
# poppler-utils — pdf2image (pdfinfo_from_path + convert_from_path)
!apt-get install -q -y ffmpeg poppler-utils
print('✅  System packages ready')

In [ ]:
# ── Cell 3: Python packages ───────────────────────────────────────────────────
# Core backend
!pip install -q \
    fastapi uvicorn[standard] \
    pydantic pydantic-settings python-dotenv \
    httpx python-multipart python-ulid \
    pdf2image Pillow \
    google-generativeai pyngrok

# Audiocraft + all its transitive deps (installed explicitly because --no-deps is used below)
!pip install -q \
    'torch>=2.1' 'torchaudio>=2.1' \
    av julius einops flashy xformers encodec \
    num2words sentencepiece huggingface_hub lameenc \
    hydra-core hydra-colorlog \
    demucs torchmetrics torchdiffeq \
    'numpy<2'

!pip install -q --no-build-isolation --no-deps \
    git+https://github.com/facebookresearch/audiocraft.git

# Verify audiocraft import in a clean subprocess (kernel cache can be stale)
import subprocess, sys
_r = subprocess.run(
    [sys.executable, '-c', 'import audiocraft; print(audiocraft.__name__)'],
    capture_output=True, text=True
)
if _r.returncode == 0:
    print('✅  audiocraft ready')
else:
    print('❌  audiocraft import failed:', _r.stderr)

In [ ]:
# ── Cell 4: Load API key + write .env ─────────────────────────────────────────
# Tries Colab Secrets first, then falls back to an environment variable.
# The key is written into /content/comikry/.env so pydantic-settings picks it up.

import os
from pathlib import Path

REPO_DIR = Path('/content/comikry')

def _load_api_key() -> str:
    # 1) Colab Secrets (🔑 left sidebar)
    try:
        from google.colab import userdata  # type: ignore
        key = userdata.get('OPENROUTER_API_KEY')
        if key:
            print('🔑  API key loaded from Colab Secrets')
            return key
    except Exception:
        pass

    # 2) Environment variable
    key = os.environ.get('OPENROUTER_API_KEY', '')
    if key:
        print('🔑  API key loaded from environment variable')
        return key

    # 3) Existing .env in the repo
    env_file = REPO_DIR / '.env'
    if env_file.exists():
        for line in env_file.read_text().splitlines():
            line = line.strip()
            if line.startswith('OPENROUTER_API_KEY=') and not line.startswith('#'):
                k = line.split('=', 1)[1].strip()
                if k and 'your_openrouter' not in k.lower():
                    print(f'🔑  API key loaded from {env_file}')
                    return k

    raise EnvironmentError(
        'OPENROUTER_API_KEY not found.\n'
        '  → Colab: left sidebar → 🔑 Secrets → Add new secret → OPENROUTER_API_KEY'
    )

OPENROUTER_API_KEY = _load_api_key()

# Write a .env that the backend settings will pick up
_env_path = REPO_DIR / '.env'
_env_path.write_text(
    f'OPENROUTER_API_KEY={OPENROUTER_API_KEY}\n'
    f'AUDIOCRAFT_DEVICE=auto\n'
    f'STORAGE_ROOT=/content/storage\n'
)
print(f'✅  .env written to {_env_path}')

In [ ]:
# ── Cell 5: Prepare storage directory ────────────────────────────────────────
import os
os.makedirs('/content/storage', exist_ok=True)
os.makedirs('/content/audiocraft_cache', exist_ok=True)
os.environ['AUDIOCRAFT_CACHE_DIR'] = '/content/audiocraft_cache'

# Change working directory to repo root so relative imports and paths resolve
os.chdir('/content/comikry')
print('✅  Working directory:', os.getcwd())
print('✅  Storage directory: /content/storage')

In [ ]:
# ── Cell 6: Start the Comikry backend server ──────────────────────────────────
# Runs uvicorn in a daemon thread so this cell returns immediately.
# The server listens on port 8000 and serves:
#   POST /comics            — upload PDF, kick off pipeline
#   GET  /comics/{id}/status — poll processing progress
#   GET  /comics/{id}/manifest — full comic JSON when done
#   GET  /storage/...       — static audio + image assets
#   GET  /                  — the Web UI (frontend/index.html)

import threading, time, subprocess, sys

_SERVER_PORT = 8000
_server_ready = threading.Event()

def _run_server():
    import uvicorn
    uvicorn.run(
        'backend.main:app',
        host='0.0.0.0',
        port=_SERVER_PORT,
        log_level='warning',
    )

_server_thread = threading.Thread(target=_run_server, daemon=True)
_server_thread.start()

# Wait until the server is accepting connections (max 30 s)
import socket
for _i in range(30):
    try:
        with socket.create_connection(('127.0.0.1', _SERVER_PORT), timeout=1):
            _server_ready.set()
            break
    except OSError:
        time.sleep(1)

if _server_ready.is_set():
    print(f'✅  Comikry backend is listening on port {_SERVER_PORT}')
else:
    print('⚠️  Server did not start within 30 s — check for import errors above')

In [ ]:
# ── Cell 7: Expose the server publicly + print Web UI link ───────────────────
#
# TWO WAYS TO USE THIS SERVER — read the output below after running:
#
#  MODE A — "Use Colab as your full backend"  (simplest)
#    Just click the Web UI link that appears below. Upload your PDF there.
#    Everything (panel detection, OCR, TTS, SFX) runs here on this Colab GPU.
#    You do NOT need to copy/paste anything into a local .env.
#
#  MODE B — "Keep your local backend, offload only SFX to Colab GPU"
#    Your local server handles everything except SFX generation, which it
#    delegates to this Colab instance for speed.
#    Add  SFX_API_URL=<url printed below>  to your local .env, then either:
#      • restart ./start.sh  (simplest), OR
#      • POST to http://localhost:8000/api/reload-config  (no restart needed)
#
# ─────────────────────────────────────────────────────────────────────────────

from IPython.display import display, HTML
import json

PUBLIC_URL = None

# ── Attempt 1: Colab built-in tunnel ─────────────────────────────────────────
try:
    from google.colab.output import eval_js  # type: ignore
    PUBLIC_URL = eval_js(f'google.colab.kernel.proxyPort({_SERVER_PORT})')
    if PUBLIC_URL:
        print('🌐  Tunnel via Colab built-in proxy')
except Exception as _e:
    print(f'ℹ️   Colab proxy unavailable ({_e}), trying ngrok …')

# ── Attempt 2: pyngrok ────────────────────────────────────────────────────────
if not PUBLIC_URL:
    try:
        from pyngrok import ngrok  # type: ignore
        # Uncomment and set your ngrok auth token for longer-lived tunnels:
        # from google.colab import userdata
        # ngrok.set_auth_token(userdata.get('NGROK_AUTH_TOKEN'))
        tunnel = ngrok.connect(_SERVER_PORT)
        PUBLIC_URL = tunnel.public_url
        print('🌐  Tunnel via pyngrok')
    except Exception as _e:
        print(f'❌  pyngrok failed: {_e}')

# ── Print results ─────────────────────────────────────────────────────────────
if PUBLIC_URL:
    PUBLIC_URL = PUBLIC_URL.replace('http://', 'https://', 1).rstrip('/')

    display(HTML(f'''
    <div style="font-family:sans-serif;border:2px solid #5b9cf6;border-radius:10px;
                padding:20px;margin:12px 0;background:#0d1117;color:#e6edf3">
      <h3 style="margin:0 0 16px;color:#5b9cf6">🎉 Comikry server is live!</h3>

      <!-- MODE A -->
      <div style="background:#161b22;border-radius:8px;padding:14px;margin-bottom:14px">
        <p style="margin:0 0 8px;font-size:14px">
          <b>✅ Mode A — Use Colab as your full backend (recommended)</b><br>
          <span style="color:#8b949e;font-size:12px">
            Click below. Upload your PDF in the browser. All processing happens here on the Colab GPU.
            Nothing to paste — it just works.
          </span>
        </p>
        <a href="{PUBLIC_URL}" target="_blank"
           style="display:inline-block;padding:10px 22px;background:#5b9cf6;
                  color:#fff;text-decoration:none;border-radius:6px;
                  font-weight:bold;font-size:15px">
          🚀 Open Comikry Web UI
        </a>
      </div>

      <!-- MODE B -->
      <div style="background:#161b22;border-radius:8px;padding:14px">
        <p style="margin:0 0 8px;font-size:14px">
          <b>⚙️ Mode B — Offload only SFX to this Colab GPU (local backend)</b><br>
          <span style="color:#8b949e;font-size:12px">
            Add the line below to your local <code>.env</code>, then either restart
            <code>./start.sh</code> <em>or</em> call the reload endpoint (no restart needed):
          </span>
        </p>
        <pre style="background:#0d1117;padding:10px;border-radius:6px;color:#79c0ff;
                    font-size:13px;margin:6px 0;overflow-x:auto">SFX_API_URL={PUBLIC_URL}</pre>
        <p style="margin:6px 0 4px;font-size:12px;color:#8b949e">
          Already running <code>./start.sh</code> locally? Activate the new URL without restarting:
        </p>
        <pre style="background:#0d1117;padding:10px;border-radius:6px;color:#79c0ff;
                    font-size:13px;margin:0;overflow-x:auto">curl -s -X POST http://localhost:8000/api/reload-config | python3 -m json.tool</pre>
      </div>

      <p style="margin:14px 0 0;font-size:11px;color:#8b949e">
        ⚠️ Keep this notebook running while using the UI.
        Colab runtimes disconnect after ~90 min of inactivity — re-run cells 6 and 7 to get a new URL.
      </p>
    </div>
    '''))

    # Also print plain-text for easy copy
    print()
    print('─' * 60)
    print('  Web UI:       ', PUBLIC_URL)
    print('  SFX_API_URL=  ', PUBLIC_URL)
    print('  Reload cmd:    curl -s -X POST http://localhost:8000/api/reload-config')
    print('─' * 60)

else:
    print()
    print('⚠️  Could not obtain a public URL.')
    print(f'   The server is running locally on port {_SERVER_PORT}')
    print('   Try manually:')
    print('     from pyngrok import ngrok')
    print(f'     print(ngrok.connect({_SERVER_PORT}).public_url)')


In [ ]:
# ── Cell 8: Tunnel auth warm-up ───────────────────────────────────────────────
# The Colab built-in tunnel shows a one-time "This notebook is connecting to an
# external server" auth page the FIRST time a new browser tab hits it.
# Until that page is dismissed, every API call from the Web UI returns an HTML
# page instead of JSON, which causes "Expecting value: line 1 column 1 (char 0)".
#
# This cell opens the bare tunnel URL in the Colab output iframe so you can
# click "Allow" once — after that the Web UI works without any errors.

from IPython.display import display, HTML, IFrame

if PUBLIC_URL:
    display(HTML(f'''
    <div style="font-family:sans-serif;border:2px solid #f59e0b;border-radius:8px;
                padding:14px;background:#1a1200;color:#fde68a;margin:8px 0">
      <b>⚠️ One-time tunnel auth step required</b><br><br>
      The Colab tunnel shows a security warning page the first time your browser
      visits it. You need to dismiss it <em>before</em> opening the Web UI,
      otherwise all API calls will fail with a JSON parse error.<br><br>
      <b>Click this link, read the warning, then click "Allow" or "Visit Site":</b><br>
      <a href="{PUBLIC_URL}/api/health" target="_blank"
         style="color:#60a5fa;font-size:14px;word-break:break-all">
        {PUBLIC_URL}/api/health
      </a><br><br>
      You should see <code style="color:#86efac">{{"ok": true}}</code> in the new tab.
      Once you see that, come back here and open the Web UI link from Cell 7.
    </div>
    '''))
else:
    print('⚠️  PUBLIC_URL not set — run Cell 7 first.')


In [ ]:
# ── Cell 8: Health check (optional) ──────────────────────────────────────────
# Confirms the server is responding correctly before you open the Web UI.

import httpx

try:
    r = httpx.get('http://127.0.0.1:8000/docs', timeout=10, follow_redirects=True)
    if r.status_code == 200:
        print('✅  Backend /docs responded with 200 — server is healthy')
        if PUBLIC_URL:
            print(f'    API docs also available at: {PUBLIC_URL.rstrip("/")}/docs')
    else:
        print(f'⚠️  Unexpected status: {r.status_code}')
except Exception as e:
    print(f'❌  Health check failed: {e}')

## ▶ How to use the Web UI

1. Click the **🚀 Open Comikry Web UI** link printed in Cell 7.
2. Drag-and-drop (or browse for) a comic PDF.
3. Optionally enable **"Normalise panel images to widescreen"**.
4. Click **Open Comic** — the PDF is sent to this Colab server for processing.
5. A progress bar shows each pipeline stage (rendering, panel detection, OCR, TTS, SFX).
6. When done, the player opens automatically.

### Player controls
| Control | Action |
|---|---|
| ▶ / ⏸ Play/Pause | Start or stop auto-playback (`Space`) |
| ◀ / ▶▶ Prev/Next | Step between panels (`←` / `→`) |
| 🔊 Voice volume | Dialogue TTS loudness |
| 🎵 SFX volume | Background sound-effect loop |
| Speed | 0.75× – 1.5× playback rate (`[` / `]`) |
| 🔗 Share | Copies a direct link to this comic |

### Keep-alive
Colab runtimes disconnect after ~90 min of inactivity. If the link stops working, re-run **Cell 6** and **Cell 7** — the server restarts and a new public URL is printed.